In [1]:

# Library Imports
import os
import import_ipynb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean, median, mode, stdev
# from Descript import extract
%matplotlib inline


In [3]:

def missed(dfName):
    # Assign global to returned DFs
    global df
    global grouped_df
    
    # Assign df to the df name provided
    df = dfName
    
    # Apply funcion used for finding missed usage dates
    def foo(gr):
        gr = gr.set_index('treatmentDate')
        idx = pd.date_range(gr.index.min(), gr.index.max())
        gr.index = pd.DatetimeIndex(gr.index)
        gr = gr.reindex(idx,  fill_value=0)
        return gr
    
    # Groupby and Apply function
    df = df.groupby(['CutitronicsClientID', 'CutitronicsSKUName']).apply(func=foo)
    
    # Groupy causes duplicate column with one as index
    # The col version is dropped to allow for reset of index
    df = df.drop(['CutitronicsClientID', 'CutitronicsSKUName'], axis = 1) 
    grouped_df = df
    df = df.reset_index()
    df =df.rename(columns={"level_2": "treatmentDate"})
    
    # Default of 0 is applied to all vlaues in missed usage row.
    # Alters comliance value to 'Missed' rather than 0
    for i, row in df.iterrows():
        if row['compliance'] == 0:
            complied = 'MISSED'
        else:
            complied = row['compliance']
        df.loc[i,'compliance'] = complied  